# *Modelo de Optimización para la Demanda de Queso Costeño*

El modelo de optimización presentado está diseñado para gestionar la demanda de queso costeño a través de una plataforma de marketplace, utilizando múltiples centros de acopio. Su objetivo principal es minimizar los costos asociados con el cumplimiento de la demanda del cliente, considerando los costos de transporte, tiempos de alistamiento y producción potencial en cada centro. El modelo busca reducir los costos totales de distribución en un sistema complejo que involucra un centro principal y varios centros secundarios, asegurando que se satisfaga la demanda total con la menor inversión en costos posible, mientras se cumplen las restricciones de capacidad y tiempo.

**Variables del Problema**

**N**: Número de centros de acopio.

**CAi**: Identificador del centro de acopio i para i=0…N donde i≠p. Estos centros de acopio complementan las unidades de producto de la demanda que el centro de acopio principal no tiene disponibles. Los centros de acopio **CAi** despachan hacia el centro de acopio principal.

**CAp**: Identificador del centro de acopio principal p (**p**∈[0,N] y **p**≠i). Este centro de acopio atiende directamente al cliente y es responsable de enviar la demanda completa al cliente.

**K(CAi)**: Cantidad del producto que se despacha desde el centro de acopio **CAi**, incluyendo unidades en stock y unidades potenciales que pueden estar listas en poco tiempo.

**Precio(CAi)**: Precio por kilo del producto despachado desde el centro de acopio **CAi**.

**cTransp(CAi)**: Costo del transporte del pedido desde el centro de acopio **CAi** a su destino.

**TiempoAlistam(CAi)**: Tiempo en horas para alistar el pedido desde el centro de acopio **CAi** a su destino.

**TiempoMaxDefinido**: Tiempo máximo definido para considerar la producción potencial.

**TiempoTransp(CAi)**: Tiempo de transporte desde el centro de acopio **CAi** a su destino.

**Tiempo(CAi)**: Tiempo total en horas para que el pedido llegue desde el centro de acopio **CAi** a su destino, sumando el tiempo de alistamiento y el tiempo de transporte.

**cTiempo**: Costo adicional por cada unidad de tiempo contemplado en la variable **Tiempo(CAi)**.

**Demanda**: Cantidad de producto solicitada por el cliente.

**Stock(CAi)**: Stock del producto en el centro de acopio **CAi**.

**Ppotencial(CAi)**: Cantidad de producto que potencialmente puede estar disponible en poco tiempo (potencial del día).

*Código desarrollado para la resolución del problema*

# IMPORTACIONES

In [91]:
from pyomo.environ import *
import pyomo.environ as pyo
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tabulate import tabulate
from prettytable import PrettyTable
import openpyxl
import time

**demanda**: Cantidad de producto de acuerdo con la categoría del pedido a satisfacer.

**costo_transporte**: Costo por unidad de tiempo de espera para satisfacer la demanda.

**id**: Identificadores de Centro de Acopio (CAi).

**kg**: Stock del Producto en el Centro de Acopio (Stock(CAi)).

**produccion_potencial**: (Ppotencial(CAi)).

**costos_transporte**: (cTransp(CAi)).

**precio**: Precio del Producto por Kilo (Precio(CAi)).

**tiempos_transporte**:Tiempos de Transporte (TiempoTransp(CAi)).

**tiempo_alistamiento**:Tiempos de Alistamiento (TiempoAlistam(CAi)).

In [92]:
centros_acopio_df = pd.read_excel('data/centros_acopio.xlsx')
tiempos_transporte_df = pd.read_excel('data/tiempos_transporte.xlsx')
costos_transporte_df = pd.read_excel('data/costo_transporte.xlsx')

# REPRESENTACIÓN DE DATOS DE ENTRADA

In [93]:
centros_acopio = centros_acopio_df.copy()
headers = ["Id_CA", "Stock", "Ppotencial", "Precio", "CtranspCli", "TiempoTranspCli", "TiempoAlistam"]
table = [[row["Id_CA"], row["Stock"], row["Ppotencial"], row["Precio"], row["CtranspCli"], row["TiempoTranspCli"], row["TiempoAlistam"]] for _, row in centros_acopio.iterrows()]

print("Centros de Acopio:")
print(tabulate(table, headers=headers, tablefmt='grid'))
print()

costo_transporte_df.columns = [''] + list(costo_transporte_df.columns[1:])
tiempos_transporte_df.columns = [''] + list(tiempos_transporte_df.columns[1:])

print("Tabla 'Costos de Transporte':")
print(tabulate(costo_transporte_df, tablefmt='grid', showindex=False, headers='keys'))
print()

table = PrettyTable()
table.field_names = costo_transporte_df.columns.tolist()
for row in costo_transporte_df.values:
    table.add_row(list(row))

print("Tabla 'Tiempos de Transporte':")
print(tabulate(tiempos_transporte_df, tablefmt='grid', showindex=False, headers='keys'))
print()

table = PrettyTable()
table.field_names = tiempos_transporte_df.columns.tolist()
for row in tiempos_transporte_df.values:
    table.add_row(list(row))

print()

Centros de Acopio:
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| Id_CA   |   Stock |   Ppotencial |   Precio |   CtranspCli |   TiempoTranspCli |   TiempoAlistam |
+=========+=========+==============+==========+==============+===================+=================+
| CA1     |      10 |        0.5   |     5000 |        21841 |                37 |              10 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA2     |      12 |        0.625 |    10000 |        21180 |                42 |               9 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA3     |       4 |       14     |    15000 |        24132 |                32 |              47 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA4     |       8 |       13     |     2000 |        14215 |          

# DEMANDA DE QUESO

In [94]:
demanda_cliente = 60

# SELECCIÓN DEL CENTRO PRINCIPAL

La función evaluar_centros_acopio analiza diversos centros de acopio utilizando criterios de costos y tiempos de transporte para identificar el que mejor se ajuste a las necesidades logísticas establecidas. Selecciona la fila con la puntuación más baja, lo que indica que es el centro de acopio más eficiente en términos de costos y tiempos de entrega.

In [121]:
def evaluar_centros_acopio(centros_acopio_df, costos_transporte_df, tiempos_transporte_df):
    """
    Evalúa los centros de acopio y devuelve el mejor centro principal basado en costos y tiempos de transporte.
    
    Parameters:
        centros_acopio_df (DataFrame): DataFrame con información sobre centros de acopio.
        costos_transporte_df (DataFrame): DataFrame con costos de transporte.
        tiempos_transporte_df (DataFrame): DataFrame con tiempos de transporte.
    
    Returns:
        str: El identificador del mejor centro de acopio.
    """
    # Asegurar que los valores sean numéricos
    costos_transporte_df = costos_transporte_df.apply(pd.to_numeric, errors='coerce')
    tiempos_transporte_df = tiempos_transporte_df.apply(pd.to_numeric, errors='coerce')

    puntuaciones = []

    # Iterar sobre cada centro de acopio como posible centro principal
    for index_centro_principal, row_centro_principal in centros_acopio_df.iterrows():
        id_centro_principal = row_centro_principal['Id_CA']
        puntuacion_total = 0

        # Evaluar los costos y tiempos desde el centro principal hacia los otros centros
        for index, row in centros_acopio_df.iterrows():
            if index == index_centro_principal:
                continue  # Saltar el centro que está siendo evaluado como principal

            # Sumar costos y tiempos de transporte
            costo_transporte = costos_transporte_df.iloc[index_centro_principal, index]
            tiempo_transporte = tiempos_transporte_df.iloc[index_centro_principal, index]

            if pd.isna(costo_transporte) or pd.isna(tiempo_transporte):
                continue  # Saltar si hay valores nulos

            puntuacion_total += costo_transporte + tiempo_transporte  # Ajusta esta fórmula según sea necesario

        # Agregar la puntuación total del centro evaluado
        puntuaciones.append((id_centro_principal, puntuacion_total))

    # Convertir a DataFrame para facilitar la selección
    df_puntuaciones = pd.DataFrame(puntuaciones, columns=['Id_CA', 'Puntuacion'])

    # Seleccionar el centro con la puntuación mínima
    mejor_centro = df_puntuaciones.loc[df_puntuaciones['Puntuacion'].idxmin()]

    return mejor_centro['Id_CA']

# Seleccionar el mejor centro de acopio sin pasar centro principal
centro_principal = evaluar_centros_acopio(centros_acopio_df, costos_transporte_df, tiempos_transporte_df)

print(f"El mejor centro de acopio seleccionado es: {centro_principal}")


El mejor centro de acopio seleccionado es: CA2


In [122]:
def obtener_costos_y_tiempos(id_centro_acopio, centro_principal, costos_transporte_df, tiempos_transporte_df):
    """
    Obtiene el costo y tiempo de transporte desde el centro principal al centro especificado.
    
    Parameters:
        id_centro_acopio (str): Identificador del centro de acopio.
        centro_principal (str): Centro de acopio principal.
        costos_transporte_df (DataFrame): DataFrame con los costos de transporte entre centros.
        tiempos_transporte_df (DataFrame): DataFrame con los tiempos de transporte entre centros.

    Returns:
        (float, float): Costo y tiempo de transporte desde el centro principal.
    """
    if id_centro_acopio == centro_principal:
        return 0, 0  # Si es el centro principal, el costo y tiempo son 0
    
    # Encontrar el índice del centro principal
    index_centro_principal = centros_acopio_df.loc[centros_acopio_df['Id_CA'] == centro_principal].index[0]
    
    # Obtener costos y tiempos de transporte
    costo_transporte = costos_transporte_df.at[index_centro_principal, id_centro_acopio]
    tiempo_transporte = tiempos_transporte_df.at[index_centro_principal, id_centro_acopio]
    
    return costo_transporte, tiempo_transporte

def agregar_fila_final(lista_filas, id_centro_acopio, cantidad, precio, costo_transporte, tiempo_transporte, tiempo_alistamiento):
    """
    Agrega una fila a la lista de datos finales con los valores calculados.

    Parameters:
        lista_filas (list): Lista para almacenar las filas del DataFrame final.
        id_centro_acopio (str): Identificador del centro de acopio.
        cantidad (int): Cantidad de producto.
        precio (float): Precio del producto.
        costo_transporte (float): Costo de transporte desde el centro principal.
        tiempo_transporte (float): Tiempo de transporte desde el centro principal.
        tiempo_alistamiento (float): Tiempo de alistamiento del producto.
    """
    lista_filas.append({
        'Id_CA': id_centro_acopio,
        'Cantidad': cantidad,
        'Precio': precio,
        'Ctransp': costo_transporte,
        'TiempoTransp': tiempo_transporte,
        'TiempoAlistam': tiempo_alistamiento
    })

def generar_dataframe_final(centro_principal):
    """
    Genera el DataFrame final que contiene la información de los centros de acopio, 
    sus cantidades de producto, precios, costos y tiempos de transporte.
    
    Parameters:
        centro_principal (str): Identificador del centro de acopio principal.

    Returns:
        DataFrame: DataFrame con los datos de centros de acopio, incluyendo transporte y tiempos.
    """
    filas_finales = []
    
    for _, row in centros_acopio_df.iterrows():
        id_centro_acopio = row['Id_CA']
        precio = row['Precio']
        
        # Obtener costos y tiempos de transporte
        costo_transporte, tiempo_transporte = obtener_costos_y_tiempos(
            id_centro_acopio, centro_principal, costo_transporte_df, tiempos_transporte_df)
        
        # Fila para el stock disponible
        cantidad_stock = 0 if id_centro_acopio == centro_principal else row['Stock']
        agregar_fila_final(filas_finales, id_centro_acopio, cantidad_stock, precio, costo_transporte, tiempo_transporte, 0)  # Tiempo alistamiento es 0 para stock
        
        # Fila para el potencial de acopio
        agregar_fila_final(filas_finales, id_centro_acopio, row['Ppotencial'], precio, costo_transporte, tiempo_transporte, row['TiempoAlistam'])
    
    return pd.DataFrame(filas_finales)

def generar_dataframes_dinamicos(centro_principal):
    """
    Genera DataFrames dinámicos para costos y tiempos de transporte con base en el centro principal.
    
    Parameters:
        centro_principal (str): Identificador del centro de acopio principal.

    Returns:
        (DataFrame, DataFrame): DataFrames de costos y tiempos de transporte.
    """
    # Encontrar el índice del centro principal
    index_centro_principal = centros_acopio_df.loc[centros_acopio_df['Id_CA'] == centro_principal].index[0]
    
    # Crear DataFrames de costos y tiempos con base en el centro principal
    costos_df = costo_transporte_df.iloc[[index_centro_principal]].reset_index(drop=True)
    tiempos_df = tiempos_transporte_df.iloc[[index_centro_principal]].reset_index(drop=True)
    
    return costos_df, tiempos_df

# Generar el DataFrame final
df_final = generar_dataframe_final(centro_principal)

# Generar DataFrames dinámicos
costos_df, tiempos_df = generar_dataframes_dinamicos(centro_principal)

# Resultados finales
print("DataFrame de Centros de Acopio:")
print(df_final)

print("\nDataFrame de Costos de Transporte:")
print(costos_df)

print("\nDataFrame de Tiempos de Transporte:")
print(tiempos_df)


DataFrame de Centros de Acopio:
   Id_CA  Cantidad  Precio  Ctransp  TiempoTransp  TiempoAlistam
0    CA1    10.000    5000    17071            35              0
1    CA1     0.500    5000    17071            35             10
2    CA2     0.000   10000        0             0              0
3    CA2     0.625   10000        0             0              9
4    CA3     4.000   15000    20339            44              0
5    CA3    14.000   15000    20339            44             47
6    CA4     8.000    2000    12897            17              0
7    CA4    13.000    2000    12897            17             41
8    CA5    18.000   30000    28898            43              0
9    CA5    15.000   30000    28898            43             42
10   CA6     6.000   50000    15196            45              0
11   CA6    14.000   50000    15196            45             16
12   CA7    12.000   15000    16108            22              0
13   CA7    13.000   15000    16108            22         